# 今日任务：数据分类 Ⅱ


sklearn 中的 datasets 方法导入训练样本，并用**留一法**产生测试样本，用 KNN 分类并输出分类精度。

【留一法：留一个样本进行测试，其它所有的用来训练，遍历所有样本]

IRIS 数据集：https://archive.ics.uci.edu/ml/datasets/iris

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# 超参数
k = 15  # k_neighbors

In [3]:

from sklearn.datasets import load_iris

# 导入 IRIS 数据集
iris = load_iris()

# 留一个样本进行测试，其它所有的用来训练，遍历所有样本
accuracy = 0
for i in range(len(iris.data)):
    X_test = np.array([iris.data[i]])
    y_test = np.array([iris.target[i]])
    X_train = np.delete(iris.data, i, axis=0)
    y_train = np.delete(iris.target, i)
    
    # 训练模型
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # 预测结果
    y_pred = knn.predict(X_test)
    
    # 输出分类精度
    accuracy += accuracy_score(y_test, y_pred)

# 输出平均分类精度
print(f"平均分类精度：{100 * accuracy / len(iris.data) :.2f}%")

CIFAR 图像数据集 https://www.cs.toronto.edu/~kriz/cifar.html

In [26]:

# 从 torch 导入 CIFAR 数据集和 vgg 模型
import torch
import os
from torchvision import datasets
from torchvision.models import vgg11
import matplotlib.pyplot as plt

# 导入 CIFAR 数据集
cifar = datasets.CIFAR10(root='datas/ml/cifar10/', train=True, download=True)
datas = cifar.data[470 : 970]              # 取出 500 张图片，索引为防抄标识
targets = cifar.targets[470 : 970]         # 取出 500 张图片的标签

# 修改 torch.hub 下载模型的路径
os.environ["TORCH_HOME"] = "models/"
torch.hub._download_url_to_file = "models/"

# 加载 vgg 模型
model = vgg11(pretrained=True)
model.eval()

# 预处理数据
data_tensor = torch.from_numpy(datas).permute(0, 3, 1, 2)   # [N, H, W, C] -> [N, C, H, W]
data_tensor = data_tensor / 255.0
data_tensor = data_tensor.float()

# 使用 vgg 模型提取特征
features = []
for i in range(len(data_tensor)):
    feature = model(data_tensor[i].unsqueeze(0))
    features.append(feature.detach().numpy())

# 留一个样本进行测试，其它所有的用来训练，遍历所有样本
accuracy = 0
for i in range(len(datas)):
    X_test = np.array([features[i]])
    y_test = np.array([targets[i]])
    X_train = np.delete(features, i, axis=0)
    y_train = np.delete(targets, i)
    
    # 训练模型
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # 预测结果
    y_pred = knn.predict(X_test)
    
    # 输出分类精度
    accuracy += accuracy_score(y_test, y_pred)

# 输出分类精度
print(f"平均分类精度：{100 * accuracy / len(datas) :.2f}%")